## Testing L2Bridge Network Services

Common configuration is in the notebook_utils/config_common.py file that is imported in the following cell. Please edit that file to config your invironment.

In [ ]:
import os
import sys
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

#import fabric_notebook_config as notebook
import fabric_notebook_utils as utils
#import fabric_vm_utils as vm_utils



## Create Slice Manager Object

In [ ]:
slice_manager = SliceManager(oc_host=utils.orchestrator_host, 
                             cm_host=utils.credmgr_host ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [ ]:
slice_name = 'MySlice_L2Bridge'
sites = [ 'STAR', 'NCSA', 'UTAH','MAX' , 'TACC']
#node1_name = 'Node1'
#node2_name = 'Node2'
#network_service_name='p2p1'
#nic1_name = 'node1-nic1'
#nic2_name = 'node2-nic1'
username='centos'
image = 'default_centos_8'
#username='ubuntu'
#image = 'default_ubuntu_20'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType

def create_l2bridge_slice(site=None, slice_name='L2bridge_test', n1_component_type=ComponentModelType.SmartNIC_ConnectX_5, n2_component_type=ComponentModelType.SmartNIC_ConnectX_5):

    node1_name = site+'Node1'
    node2_name = site+'Node2'
    nic1_name = node1_name+'-nic1'
    nic2_name = node2_name+'-nic1'
    
    
    # Create topology
    t = ExperimentTopology()

    # Add node
    n1 = t.add_node(name=node1_name, site=site)

    # Set capacities
    cap = Capacities()
    cap.set_fields(core=cores, ram=ram, disk=disk)

    # Set Properties
    n1.set_properties(capacities=cap, image_type=image_type, image_ref=image)

    # Add node
    n2 = t.add_node(name=node2_name, site=site)

    # Set properties
    n2.set_properties(capacities=cap, image_type=image_type, image_ref=image)

    # Shared Cards
    n1.add_component(model_type=n1_component_type, name=nic1_name)
    n2.add_component(model_type=n2_component_type, name=nic2_name)

    if_labels = n1.interface_list[0].get_property(pname="labels")
    if_labels.vlan = "200"
    n1.interface_list[0].set_properties(labels=if_labels)
    if_labels = n2.interface_list[0].get_property(pname="labels")
    if_labels.vlan = "200"
    n2.interface_list[0].set_properties(labels=if_labels)

    # L2bridge Service
    t.add_network_service(name='br1', nstype=ServiceType.L2Bridge,
                      interfaces=[n1.interface_list[0], n2.interface_list[0]])


    # Generate Slice Graph
    slice_graph = t.serialize()

    # Request slice from Orchestrator
    return_status, slice_reservations = slice_manager.create(slice_name=slice_name, 
                                                slice_graph=slice_graph, 
                                                ssh_key=utils.ssh_key_pub)

    if return_status == Status.OK:
        slice_id = slice_reservations[0].get_slice_id()
        print("Submitted slice creation request. Slice ID: {}".format(slice_id))
    else:
        print(f"Failure: {slice_reservations}")
        
    return slice_id

def print_nodes(verbose=False):
    return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

    for node_name, node in experiment_topology.nodes.items():
        if verbose:
            print("Node:")
            print("   Name              : {}".format(node.name))
            print("   Cores             : {}".format(node.get_property(pname='capacity_allocations').core))
            print("   RAM               : {}".format(node.get_property(pname='capacity_allocations').ram))
            print("   Disk              : {}".format(node.get_property(pname='capacity_allocations').disk))
            print("   Image             : {}".format(node.image_ref))
            print("   Image Type        : {}".format(node.image_type))
            print("   Host              : {}".format(node.get_property(pname='label_allocations').instance_parent))
            print("   Site              : {}".format(node.site))
            print("   Management IP     : {}".format(node.management_ip))
            print("   Reservation ID    : {}".format(node.get_property(pname='reservation_info').reservation_id))
            print("   Reservation State : {}".format(node.get_property(pname='reservation_info').reservation_state))
            print("   Components        : {}".format(node.components))
            print("   Interfaces        : {}".format(node.interfaces))
            print()    
        else:
            print("{}: manegement_ip: {}".format(node.name,node.management_ip))
        
def configure_nodes():
    return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)   

    for node_name, node in experiment_topology.nodes.items():
        management_ip = str(node.management_ip)

        stdout = utils.execute_script(username, node, 'sudo yum install -q -y python3 python3-netifaces')
        #print("stdout: {}".format(stdout))

        file_attributes = utils.upload_file(username, node, 'fabric_vm_utils.py','fabric_vm_utils.py')
        #print("file_attributes: {}".format(file_attributes))
        
def run_tests(node1_name,node2_name):
    import ast

    return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)   


    #for node_name, node in experiment_topology.nodes.items():
    #Node1
    node1 = experiment_topology.nodes[node1_name]
    node_ip = '192.168.1.1'
    node_cidr = '24'
    management_ip = str(node1.management_ip)

    print("{}: manegement_ip: {}".format(node1.name,node1.management_ip), end='')
    stdout = utils.execute_script(username, node1, 'sudo python3 fabric_vm_utils.py {} {} {}'.format('get_management_iface',node_ip,node_cidr) )
    print(", management_iface: {}".format(stdout), end='')

    stdout = utils.execute_script(username, node1, 'sudo python3 fabric_vm_utils.py {} {} {}'.format('get_datalane_ifaces',node_ip,node_cidr) )
    #print("stdout: {}".format(stdout))
    dataplane_ifaces = ast.literal_eval(stdout) 
    print("dataplane_ifaces: {}".format(dataplane_ifaces))
    #for iface in dataplane_ifaces:
        #print("dataplane iface: {}".format(iface))
        #stdout = utils.execute_script('ubuntu', node, 'sudo python3 fabric_vm_utils.py {} {} {}'.format('delete_vlan_iface', iface, 200) )

    
    #Node2
    node2 = experiment_topology.nodes[node2_name]
    node_ip = '192.168.1.2'
    node_cidr = '24'
    management_ip = str(node2.management_ip)

    print("{}: manegement_ip: {}".format(node2.name,node2.management_ip), end='')
    stdout = utils.execute_script(username, node2, 'sudo python3 fabric_vm_utils.py {} {} {}'.format('get_management_iface',node_ip,node_cidr) )
    print(", management_iface: {}".format(stdout), end='')

    stdout = utils.execute_script(username, node2, 'sudo python3 fabric_vm_utils.py {} {} {}'.format('get_datalane_ifaces',node_ip,node_cidr) )
    #print("stdout: {}".format(stdout))
    dataplane_ifaces = ast.literal_eval(stdout) 
    print("dataplane_ifaces: {}".format(dataplane_ifaces))
    #for iface in dataplane_ifaces:
        #print("dataplane iface: {}".format(iface))
        #stdout = utils.execute_script('ubuntu', node, 'sudo python3 fabric_vm_utils.py {} {} {}'.format('delete_vlan_iface', iface, 200) )
   

    #Find Mapping
    utils.find_nic_mapping(node1=node1, node1_username=username, node1_vlan='200',  
                           node2=node2, node2_username=username, node2_vlan='200')




In [ ]:
import time

for site in sites:
   
    slice_name='L2birdgeTest_{}'.format(site)
    print("XXXXXXXXXXXXXXXXXX L2bridge  Test: {} XXXXXXXXXXXXXXXXXX".format(site))
    try:
        create_l2bridge_slice(site=site, 
                              slice_name=slice_name, 
                              n1_component_type=ComponentModelType.SmartNIC_ConnectX_5, 
                              n2_component_type=ComponentModelType.SmartNIC_ConnectX_5)


        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            slice = utils.wait_for_slice(slice, timeout=300, progress=True, slice_manager=slice_manager)

        print()
        print("Slice Name : {}".format(slice.slice_name))
        print("ID         : {}".format(slice.slice_id))
        print("State      : {}".format(slice.slice_state))
        print("Lease End  : {}".format(slice.lease_end))

        time.sleep(30)

        node1_name = site+'Node1'
        node2_name = site+'Node2'

        print_nodes()  
        configure_nodes()
        run_tests(node1_name,node2_name)

    except Exception as e:
        print("Error Running Test: {}".format(str(e)))
        
    try:
        #Delete Slice
        return_status, result = slice_manager.delete(slice_object=slice)

        print("Response Status {}".format(return_status))
        print("Response received {}".format(result))
        time.sleep(10)
    except Exception as e:
        print("Error Deleting Slice: {}".format(str(e)))
   

## Delete Slice

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))